# Clean & Merge/Match 3 Datasets

In [3]:
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [4]:
df = pd.read_csv(r"C:\Users\Admin\Documents\Studio Code\customer_retention_data\zoom_logs.csv")
df.shape

(277988, 23)

In [5]:
#Extract 5-digit SID from log in zoom name in case the student does not log in with verified school email
mask = (df['role'] == 'Student') & (df['SID_str'].isna())
extracted = (
    df.loc[mask, 'Name (original name)']
      .astype(str)
      .str.extract(r'(?<!\d)(\d{5}|\d{8})(?!\d)', expand=False)
)

# Assign only where we actually found a match
df.loc[mask, 'SID_str'] = extracted

#Check the newly extracted rows
df1 = df.loc[mask]
df1.head()

,Topic,ID,Host name,Host email,Start time,End time,Participants,Duration (minutes),Name (original name),Email,...,Guest,SID,date,time,day_of_week,hour,stu_join_time,stu_leave_time,SID_str,role
1,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,"Hoang Anh, Bao",NaN,...,Yes,NaN,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,NaN,Student
9,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,#Nguyen Tam Doan 04114 ( Nguyen Tam Doan 04114 ),NaN,...,Yes,NaN,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,04114,Student
13,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Thao Nguyen Nguyen,NaN,...,Yes,NaN,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,NaN,Student
19,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,"#Nguyen Hong Hanh Nhien 00790 ( ""Nguyen Nhien,...",NaN,...,Yes,NaN,9/30/2024,19:25:00,Monday,19,19:26:00,20:25:00,00790,Student
21,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,**Phạm Ngọc Kim Ngân 04410 ( Phạm Ngọc Kim Ngâ...,NaN,...,Yes,NaN,9/30/2024,19:25:00,Monday,19,19:26:00,20:25:00,04410,Student


In [6]:
#CHange SID column from float to str without 0s at the bback
df['SID'] = df['SID'].astype('object')
df.dtypes

Topic                    object
ID                       object
Host name                object
Host email               object
Start time               object
End time                 object
Participants              int64
Duration (minutes)        int64
Name (original name)     object
Email                    object
Join time                object
Leave time               object
student_stay_duration     int64
Guest                    object
SID                      object
date                     object
time                     object
day_of_week              object
hour                      int64
stu_join_time            object
stu_leave_time           object
SID_str                  object
role                     object
dtype: object

In [7]:
# Identify rows where SID_str is exactly 8 digits (no letters or symbols)
mask_8 = df['SID_str'].astype(str).str.match(r'^\d{8}$', na=False)

# Transfer only those 8-digit numeric strings to SID
df.loc[mask_8, 'SID'] = df.loc[mask_8, 'SID_str']

# (Optional) Clear them from SID_str after moving
# df.loc[mask_8, 'SID_str'] = pd.NA


In [8]:
df['SID_5digit'] = df['SID_str'].astype(str).str.extract(r'(\d{5})$')
df.head()

,Topic,ID,Host name,Host email,Start time,End time,Participants,Duration (minutes),Name (original name),Email,...,SID,date,time,day_of_week,hour,stu_join_time,stu_leave_time,SID_str,role,SID_5digit
0,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,TA Phung ( schedule 1 ) (Host),schedule1@minhvietacademy.org,...,NaN,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,schedule1,TA/Teacher,NaN
1,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,"Hoang Anh, Bao",NaN,...,NaN,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,NaN,Student,NaN
2,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Mai Nguyen Truc Linh 01816,23001816@minhvietacademy.org,...,23001816,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,23001816,Student,01816
3,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Nguyen Pham Nhat Minh 02469,22002469@minhvietacademy.org,...,22002469,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,22002469,Student,02469
4,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Yen Ha Tu Vy 02190,24002190@minhvietacademy.org,...,24002190,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,24002190,Student,02190


#### Match Grade

In [9]:
#read the topic_class names dictionary to match the zoom classes to the right Grade
df_grade_class_dict = pd.read_excel(r"C:\Users\Admin\Documents\Studio Code\customer_retention_data\goitenemmai\SY25_MVA_Goitenemmai (1).xlsx", sheet_name="grade_class_dict")

df_grade_class_dict

,Topic,Grade
0,2500COM,Grade K
1,2501HUM,Grade 1
2,2502HUM,Grade 2
3,2503HUM,Grade 3
4,2504HUM,Grade 4
5,2505HUM,Grade 5
6,2501MAT,Grade 1
7,2502MAT,Grade 2
8,2503MAT,Grade 3
9,2504MAT,Grade 4


In [10]:
df_merged = pd.merge(df, df_grade_class_dict, on="Topic", how="left")
df_merged.head()

,Topic,ID,Host name,Host email,Start time,End time,Participants,Duration (minutes),Name (original name),Email,...,date,time,day_of_week,hour,stu_join_time,stu_leave_time,SID_str,role,SID_5digit,Grade
0,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,TA Phung ( schedule 1 ) (Host),schedule1@minhvietacademy.org,...,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,schedule1,TA/Teacher,NaN,Grade 1
1,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,"Hoang Anh, Bao",NaN,...,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,NaN,Student,NaN,Grade 1
2,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Mai Nguyen Truc Linh 01816,23001816@minhvietacademy.org,...,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,23001816,Student,01816,Grade 1
3,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Nguyen Pham Nhat Minh 02469,22002469@minhvietacademy.org,...,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,22002469,Student,02469,Grade 1
4,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Yen Ha Tu Vy 02190,24002190@minhvietacademy.org,...,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,24002190,Student,02190,Grade 1


### Match student list

In [35]:
#read the list of students dataset
df_students = pd.read_excel(r"C:\Users\Admin\Documents\Studio Code\customer_retention_data\goitenemmai\SY25_MVA_Goitenemmai (1).xlsx", dtype={'SID_5digit': str, 'SID_8digit': str})
df_students.shape

(2343, 12)

#### Match with those have full SIDs first

In [36]:
#Find the list of students on zoom (df) that doesnt match with any in the official Student list
m = pd.merge(
    df_merged,
    df_students,
    left_on='SID',
    right_on='SID_8digit',
    how='left',
    indicator=True
)
m.shape

(277988, 38)

#### Find the list of students that don't match

In [39]:
zoom_only = m[m['_merge'] == 'left_only']
zoom_only.shape
zoom_only.to_excel('zoom_only.xlsx', index=False)

In [45]:
matched_8 = m[m['_merge'] == 'both']
matched_8.shape
#m.to_excel('results_8digit.xlsx', index=False)

(233455, 38)

In [51]:
students_only = m[m['_merge'] == 'right_only']
students_only.shape
#zoom_only.to_excel('zoom_only.xlsx', index=False)

(0, 38)

#### Match those only have SID 5 digits

In [42]:
zoom_5digits = pd.read_excel(r"C:\Users\Admin\Documents\Studio Code\customer_retention_project\zoom_only.xlsx", dtype={'SID_5digit': str, 'SID':str})
zoom_5digits

,Topic,ID,Host name,Host email,Start time,End time,Participants,Duration (minutes),Name (original name),Email,...,date,time,day_of_week,hour,stu_join_time,stu_leave_time,SID_str,role,SID_5digit,Grade
0,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,TA Phung ( schedule 1 ) (Host),schedule1@minhvietacademy.org,...,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,schedule1,TA/Teacher,NaN,Grade 1
1,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,"Hoang Anh, Bao",NaN,...,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,NaN,Student,NaN,Grade 1
2,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,#Nguyen Tam Doan 04114 ( Nguyen Tam Doan 04114 ),NaN,...,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,04114,Student,04114,Grade 1
3,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Thao Nguyen Nguyen,NaN,...,9/30/2024,19:25:00,Monday,19,19:25:00,20:25:00,NaN,Student,NaN,Grade 1
4,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,"#Nguyen Hong Hanh Nhien 00790 ( ""Nguyen Nhien,...",NaN,...,9/30/2024,19:25:00,Monday,19,19:26:00,20:25:00,00790,Student,00790,Grade 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44528,25APBIO,942 5158 6913,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 19:25,5/5/2025 20:23,47,58,"Nguyen Tien, Anh",NaN,...,5/5/2025,19:25:59,Monday,19,19:38:36,20:23:42,NaN,Student,NaN,Grade 9-12
44529,25APENG,996 9559 8473,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 18:24,5/5/2025 19:25,40,61,TA Bang ( Schedule 9 ) (Host),schedule9@minhvietacademy.org,...,5/5/2025,18:24:38,Monday,18,18:24:38,19:25:37,schedule9,TA/Teacher,NaN,Grade 9-12
44530,25APENG,996 9559 8473,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 18:24,5/5/2025 19:25,40,61,Kandi Bartling,kbartling@minhvietacademy.org,...,5/5/2025,18:24:38,Monday,18,18:30:37,19:25:29,kbartling,TA/Teacher,NaN,Grade 9-12
44531,25APENG,996 9559 8473,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 18:24,5/5/2025 19:25,40,61,phạm nguyên phúc hưng 02068,NaN,...,5/5/2025,18:24:38,Monday,18,18:37:45,19:25:37,02068,Student,02068,Grade 9-12


In [43]:
#Find the list of students on zoom (df) that doesnt match with any in the official Student list
m_5digits = pd.merge(
    zoom_5digits,
    df_students,
    left_on=['SID_5digit', 'Grade'],
    right_on=['SID_5digit', 'Grade_bytopic'],
    how='left',
    indicator=True
)
m_5digits.shape

(44598, 37)

In [ ]:
not_matched_5 = m_5digits[m_5digits['_merge'] == 'left_only']
not_matched_5.shape #these are the ones that cannot match their SID_5digit+Grade with any student profile / blank SID 
#not_matched_5.to_excel('not_matched_5digit.xlsx', index=False) #~1230 students (excluded TAs and Teachers)

(33382, 37)

In [47]:
matched_5 = m_5digits[m_5digits['_merge'] == 'both']
matched_5.to_excel('matched_5digit.xlsx', index=False)

#### Concat the matched (8digits) with the unmatched_8digits back

In [48]:
matched_8 = matched_8.drop(columns=['SID_5digit_y'])
matched_8 = matched_8.rename(columns={'SID_5digit_x': 'SID_5digit'})
matched_8.columns

Index(['Topic', 'ID', 'Host name', 'Host email', 'Start time', 'End time',
       'Participants', 'Duration (minutes)', 'Name (original name)', 'Email',
       'Join time', 'Leave time', 'student_stay_duration', 'Guest', 'SID_x',
       'date', 'time', 'day_of_week', 'hour', 'stu_join_time',
       'stu_leave_time', 'SID_str', 'role', 'SID_5digit', 'Grade_x', 'SID_y',
       'SID_8digit', 'Name', 'Grade_y', 'Grade_bytopic', 'EFL', 'Note',
       'MVA/MVSM', 'concat', '4 last digits',
       'Học sinh cùng lớp có chung 4 số cuối ID', '_merge'],
      dtype='object')

In [47]:
m_5digits.columns

Index(['Topic', 'ID', 'Host name', 'Host email', 'Start time', 'End time',
       'Participants', 'Duration (minutes)', 'Name (original name)', 'Email',
       'Join time', 'Leave time', 'student_stay_duration', 'Guest', 'SID_x',
       'date', 'time', 'day_of_week', 'hour', 'stu_join_time',
       'stu_leave_time', 'SID_str', 'role', 'SID_5digit', 'Grade_x', 'SID_y',
       'SID_8digit', 'Name', 'Grade_y', 'Grade_bytopic', 'EFL', 'Note',
       'MVA/MVSM', 'concat', '4 last digits',
       'Học sinh cùng lớp có chung 4 số cuối ID', '_merge'],
      dtype='object')

In [49]:
df_combined = pd.concat([matched_8,m_5digits], ignore_index=True)
df_combined
df_combined.to_excel('combined.xlsx', index=False)

## Final Dataset

In [56]:
df_final = pd.read_excel(r"C:\Users\Admin\Documents\Studio Code\customer_retention_project\combined.xlsx")
df_final

,Topic,ID,Host name,Host email,Start time,End time,Participants,Duration (minutes),Name (original name),Email,...,SID_full,Name,Grade,EFL,Note,MVA/MVSM,concat,4 last digits,Học sinh cùng lớp có chung 4 số cuối ID,_merge
0,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Mai Nguyen Truc Linh 01816,23001816@minhvietacademy.org,...,2300-1816,Mai Nguyễn Trúc Linh,Grade 1,EFL K5,Chính thức,MVA,MVA2300-1816,1816.0,NaN,both
1,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Nguyen Pham Nhat Minh 02469,22002469@minhvietacademy.org,...,2200-2469,Nguyễn Phạm Nhật Minh,Grade 1,EFL K5,Chính thức,MVA,MVA2200-2469,2469.0,NaN,both
2,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Yen Ha Tu Vy 02190,24002190@minhvietacademy.org,...,2400-2190,VY HÀ TÚ YÊN,Grade 1,EFL K5,Chính thức,MVA,MVA2400-2190,2190.0,NaN,both
3,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Lai Ngan Khanh 01771,23001771@minhvietacademy.org,...,2300-1771,Lại Ngân Khánh,Grade 1,EFL K5,Chính thức,MVA,MVA2300-1771,1771.0,NaN,both
4,2501HUM,977 3083 0512,schedule 1,schedule1@minhvietacademy.org,9/30/2024 19:25,9/30/2024 20:25,211,60,Tran Minh Khang 03927,23003927@minhvietacademy.org,...,2300-3927,Trần Minh Khang,Grade 1,EFL K5,Chính thức,MVA,MVA2300-3927,3927.0,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278048,25APBIO,942 5158 6913,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 19:25,5/5/2025 20:23,47,58,"Nguyen Tien, Anh",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
278049,25APENG,996 9559 8473,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 18:24,5/5/2025 19:25,40,61,TA Bang ( Schedule 9 ) (Host),schedule9@minhvietacademy.org,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
278050,25APENG,996 9559 8473,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 18:24,5/5/2025 19:25,40,61,Kandi Bartling,kbartling@minhvietacademy.org,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
278051,25APENG,996 9559 8473,Schedule 9,schedule9@minhvietacademy.org,5/5/2025 18:24,5/5/2025 19:25,40,61,phạm nguyên phúc hưng 02068,NaN,...,2400-2068,Phạm Nguyên Phúc Hưng,Grade 9-12,EFL 69,Chính thức,MVA,MVA2400-2068,2068.0,NaN,both


In [61]:
df_final1 = df_final.drop(
    df_final[
       (df_final['SID'].isna() |(df_final['SID']=='')) &
       (df_final['_merge']=='left_only') &
       (df_final['role']=='Student')
    ].index
)

df_final1.shape

(256585, 35)

In [62]:
df_final1.to_excel('combined_student_zoom.xlsx', index=None)

## Canvas Quiz Files Merging

In [67]:
import glob
import os

# 📂 Path to your Excel folder
folder_path = r"C:\Users\Admin\Documents\Studio Code\customer_retention_data\canvas"

# 🔍 Find all Excel files
files = glob.glob(os.path.join(folder_path, "*.csv"))

all_long = []

for f in files:
    # ---- 1️⃣ Read Excel file, skipping subheader row ----
    df = pd.read_csv(f, skiprows=[1])  # skip the 2nd row (index=1)
    
    # ---- 2️⃣ Clean column names ----
    df.columns = df.columns.str.strip()
    
    # ---- 3️⃣ Keep useful ID columns only if they exist ----
    id_cols = [c for c in ['ID', 'SIS Login ID', 'Student','Section'] if c in df.columns]
    
    # ---- 4️⃣ Melt into long (tidy) format ----
    df_long = df.melt(
        id_vars=id_cols,
        var_name='Quiz',
        value_name='Score'
    )
    
    # ---- 5️⃣ Tag source (class or filename) ----
    class_name = os.path.splitext(os.path.basename(f))[0]
    df_long['Source_File'] = class_name
    
    all_long.append(df_long)

# ---- 6️⃣ Combine all files ----
df_all = pd.concat(all_long, ignore_index=True)

# ---- 7️⃣ Clean up ----
df_all['Score'] = pd.to_numeric(df_all['Score'], errors='coerce')

# Optional: extract topic names without dates or IDs in parentheses
df_all['Quiz_Clean'] = df_all['Quiz'].str.replace(r'\s*\(.*?\)', '', regex=True).str.strip()

# ---- 8️⃣ Save master dataset ----
print("✅ Combined shape:", df_all.shape)
print(df_all.head())


✅ Combined shape: (397226, 8)
      ID SIS Login ID                        Student                  Section  \
0  10424     23003601      Bui Hoang Bach - 23003601  Grade 8 Algebra I F2024   
1   3336     20034287  Bùi Lê Thảo Nguyên - 20034287  Grade 8 Algebra I F2024   
2   9599     22001282   Bùi Trần Bảo Minh - 22001282  Grade 8 Algebra I F2024   
3   2986     20043685     Bùi Vũ Gia Linh - 20043685  Grade 8 Algebra I F2024   
4  10863     23000899       Cao Quỳnh Lam - 23000899  Grade 8 Algebra I F2024   

                           Quiz  Score  \
0  1.1 Linear Functions (78170)  100.0   
1  1.1 Linear Functions (78170)    NaN   
2  1.1 Linear Functions (78170)  100.0   
3  1.1 Linear Functions (78170)  100.0   
4  1.1 Linear Functions (78170)    NaN   

                                      Source_File            Quiz_Clean  
0  2025-09-06T0348_Grades-Grade_8_Algebra_I_F2024  1.1 Linear Functions  
1  2025-09-06T0348_Grades-Grade_8_Algebra_I_F2024  1.1 Linear Functions  
2  2025-

In [68]:
df_all.tail(10)

,ID,SIS Login ID,Student,Section,Quiz,Score,Source_File,Quiz_Clean
397216,11635,24001398,Võ Thục Quyên - 24001398,Grade K Spring 2025,Unposted Final Score,94.64,2025-09-06T0406_Grades-Grade_K_SP2025,Unposted Final Score
397217,10946,24000768,Vũ An Nhiên - 24000768,Grade K Spring 2025,Unposted Final Score,100.00,2025-09-06T0406_Grades-Grade_K_SP2025,Unposted Final Score
397218,12186,24002461,Vũ Ánh Dương - 24002461,Grade K Spring 2025,Unposted Final Score,98.55,2025-09-06T0406_Grades-Grade_K_SP2025,Unposted Final Score
397219,11296,24001099,Vũ Gia Nguyên - 24001099,Grade K Spring 2025,Unposted Final Score,91.73,2025-09-06T0406_Grades-Grade_K_SP2025,Unposted Final Score
397220,11268,24001050,Vũ Khánh Linh - 24001050,Grade K Spring 2025,Unposted Final Score,98.55,2025-09-06T0406_Grades-Grade_K_SP2025,Unposted Final Score
397221,12387,25000385,Vũ Kỳ Thanh - 25000385,Grade K Spring 2025,Unposted Final Score,21.82,2025-09-06T0406_Grades-Grade_K_SP2025,Unposted Final Score
397222,12202,23004064,Vũ Lam Cát Tường - 23004064,Grade K Spring 2025,Unposted Final Score,0.00,2025-09-06T0406_Grades-Grade_K_SP2025,Unposted Final Score
397223,12151,24002269,Vũ Ngọc Lê Minh - 24002269,Grade K Spring 2025,Unposted Final Score,38.91,2025-09-06T0406_Grades-Grade_K_SP2025,Unposted Final Score
397224,12131,24002325,Vũ Trần Trâm Anh - 24002325,Grade K Spring 2025,Unposted Final Score,91.27,2025-09-06T0406_Grades-Grade_K_SP2025,Unposted Final Score
397225,12199,5374d4f4c8ac2dd67f54d76cacc0b9d3ac3d8508,"Student, Test",Grade K Spring 2025,Unposted Final Score,5.82,2025-09-06T0406_Grades-Grade_K_SP2025,Unposted Final Score


In [69]:
df_all.to_excel("all_quizzes_long.xlsx", index=False)